In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from torch.utils.data import Dataset, random_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import codecs
from dataloaders import *
from processor import *

In [3]:
# Setting random seed and device
SEED = 1

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

/home/patrick/Documents/NLP/CW/funny-headlines/venv/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:
train_df = pd.read_csv('data/task-1/train.csv')
test_df = pd.read_csv('data/task-1/dev.csv')

training_data = train_df['original']
training_edits = train_df['edit']
test_data = test_df['original']
test_edits = test_df['edit']

edited_training = pd.Series(create_edited_sentences(training_data, training_edits))
edited_test = pd.Series(create_edited_sentences(test_data, test_edits))

# Creating word vectors
training_vocab, training_tokenized_corpus = create_vocab(edited_training)
test_vocab, test_tokenized_corpus = create_vocab(edited_test)
joint_vocab, joint_tokenized_corpus = create_vocab(pd.concat([edited_training, edited_test]))

training_grades = train_df['meanGrade']

training_vector_sentences = vectorize_sentences(training_tokenized_corpus, joint_vocab)
training_dataset = Task1Dataset(training_vector_sentences, training_grades)

train_dataset, validation_dataset = dataset_split(training_dataset)

BATCH_SIZE = 32
train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE, collate_fn=collate_fn_padd)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn_padd)

glove_tensor = build_embedding_tensor(joint_vocab, 100)

In [5]:
def model_performance(output, target, print_output=False):
    """
    Returns SSE and MSE per batch (printing the MSE and the RMSE)
    """

    sq_error = (output - target)**2

    sse = np.sum(sq_error)
    mse = np.mean(sq_error)
    rmse = np.sqrt(mse)

    if print_output:
        print(f'| MSE: {mse:.2f} | RMSE: {rmse:.2f} |')

    return sse, mse


In [6]:
class FFNN(nn.Module):
    def __init__(self, glove_tensor, embedding_dim, vocab_size):
        super(FFNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0).from_pretrained(glove_tensor)
        # hidden layer
        self.fc1 = nn.Linear(embedding_dim, 20)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(20, 10)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(10, 5)
        self.relu3 = nn.ReLU()
        # output layer
        self.output = nn.Linear(5, 1)

    def forward(self, x):
        embedded = self.embedding(x)
        sentence_lengths = x.ne(0).sum(1, keepdims=True)
        averaged = embedded.sum(1) / sentence_lengths
        out = self.fc1(averaged)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.output(out)
        return out

In [15]:
class CNN(nn.Module):
  def __init__(self, glove, vocab_size, embedding_dim, out_channels, window_size, output_dim, dropout):
    super(CNN, self).__init__()
    
    # Create the embedding layer as usual
    self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0).from_pretrained(glove)
    
    self.conv = nn.Conv2d(
      in_channels=1, out_channels=out_channels,
      kernel_size=(window_size, embedding_dim))
    
    # the dropout layer
    self.dropout = nn.Dropout(dropout)

    # the output layer
    self.fc = nn.Linear(out_channels, output_dim)
        
  def forward(self, x):
    # x -> (batch size, max_sent_length)
    
    embedded = self.embedding(x)
    # embedded -> (batch size, max_sent_length, embedding_dim)
    
    # images have 3 RGB channels 
    # for the text we add 1 channel
    embedded = embedded.unsqueeze(1)
    # embedded -> (batch size, 1, max_sent_length, embedding dim)

    # Compute the feature maps      
    feature_maps = self.conv(embedded)

    ##########################################
    # Q: What is the shape of `feature_maps` ?
    ##########################################
    # feature_maps -> (batch_size, out_channels, max_sent_len - window_size + 1, 1)
    
    feature_maps = feature_maps.squeeze(3)
    
    ##########################################
    # Q: why do we remove 1 dimension here?
    ##########################################
    # The last dimension is not needed as it is of size 1
    
    # Apply ReLU
    feature_maps = F.relu(feature_maps)
    
    # Apply the max pooling layer
    pooled = F.max_pool1d(feature_maps, feature_maps.shape[2])
    
    pooled = pooled.squeeze(2)

    ####################################
    # Q: What is the shape of `pooled` ?
    ####################################
    # pooled -> (batch_size, out_channels)
    
    dropped = self.dropout(pooled)
    preds = self.fc(dropped)
    
    return preds

In [7]:
def eval(data_iter, model):
    """
    Evaluating model performance on the dev set
    """
    model.eval()
    epoch_loss = 0
    epoch_sse = 0
    pred_all = []
    trg_all = []
    no_observations = 0

    with torch.no_grad():
        for batch in data_iter:
            feature, target = batch
            feature, target = feature.to(device), target.to(device)
            # for RNN:
            # model.batch_size = target.shape[0]
            no_observations = no_observations + target.shape[0]
            # model.hidden = model.init_hidden()

            predictions = model(feature).squeeze(1)
            loss = loss_fn(predictions, target)

            # We get the mse
            pred, trg = predictions.detach().cpu().numpy(), target.detach().cpu().numpy()
            sse, __ = model_performance(pred, trg)

            epoch_loss += loss.item()*target.shape[0]
            epoch_sse += sse
            pred_all.extend(pred)
            trg_all.extend(trg)

    return epoch_loss/no_observations, epoch_sse/no_observations, np.array(pred_all), np.array(trg_all)

In [29]:
EMBEDDING_DIM = 100
HIDDEN_DIM = 50
# model = FFNN(glove_tensor, EMBEDDING_DIM, len(joint_vocab))
model = CNN(glove_tensor, len(joint_vocab), EMBEDDING_DIM, 3, 5, 1, 0.2)
optimizer = optim.Adam(model.parameters())
loss_fn = nn.MSELoss()

def train(train_loader, validation_loader, model, number_epoch):
    """
    Training loop for the model, which calls on eval to evaluate after each epoch
    """
    print("Training model.")
    for epoch in range(1, number_epoch+1):
        model.train()
        epoch_loss = 0
        epoch_sse = 0
        no_observations = 0  # Observations used for training so far
        for batch in train_loader:
            feature, target = batch
            feature, target = feature.to(device), target.to(device)
            # for RNN:
            # model.batch_size = target.shape[0]
            no_observations = no_observations + target.shape[0]
            # model.hidden = model.init_hidden()
            predictions = model(feature).squeeze(1)
            optimizer.zero_grad()
            loss = loss_fn(predictions, target)
            sse, __ = model_performance(predictions.detach().cpu().numpy(), target.detach().cpu().numpy())
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()*target.shape[0]
            epoch_sse += sse

        valid_loss, valid_mse, __, __ = eval(validation_loader, model)

        epoch_loss, epoch_mse = epoch_loss / no_observations, epoch_sse / no_observations
        print(f'| Epoch: {epoch:02} | Train Loss: {epoch_loss:.2f} | Train MSE: {epoch_mse:.2f} | Train RMSE: {epoch_mse**0.5:.2f} | \
        Val. Loss: {valid_loss:.2f} | Val. MSE: {valid_mse:.2f} |  Val. RMSE: {valid_mse**0.5:.2f} |')

In [31]:
train(train_loader, validation_loader, model, 200)

Training model.
| Epoch: 01 | Train Loss: 0.28 | Train MSE: 0.28 | Train RMSE: 0.53 |         Val. Loss: 0.33 | Val. MSE: 0.33 |  Val. RMSE: 0.57 |
| Epoch: 02 | Train Loss: 0.28 | Train MSE: 0.28 | Train RMSE: 0.53 |         Val. Loss: 0.33 | Val. MSE: 0.33 |  Val. RMSE: 0.57 |
| Epoch: 03 | Train Loss: 0.28 | Train MSE: 0.28 | Train RMSE: 0.53 |         Val. Loss: 0.33 | Val. MSE: 0.33 |  Val. RMSE: 0.58 |
| Epoch: 04 | Train Loss: 0.28 | Train MSE: 0.28 | Train RMSE: 0.53 |         Val. Loss: 0.33 | Val. MSE: 0.33 |  Val. RMSE: 0.57 |
| Epoch: 05 | Train Loss: 0.28 | Train MSE: 0.28 | Train RMSE: 0.53 |         Val. Loss: 0.33 | Val. MSE: 0.33 |  Val. RMSE: 0.58 |
| Epoch: 06 | Train Loss: 0.28 | Train MSE: 0.28 | Train RMSE: 0.53 |         Val. Loss: 0.33 | Val. MSE: 0.33 |  Val. RMSE: 0.57 |
| Epoch: 07 | Train Loss: 0.28 | Train MSE: 0.28 | Train RMSE: 0.53 |         Val. Loss: 0.33 | Val. MSE: 0.33 |  Val. RMSE: 0.58 |
| Epoch: 08 | Train Loss: 0.28 | Train MSE: 0.28 | Train RMS

| Epoch: 63 | Train Loss: 0.26 | Train MSE: 0.26 | Train RMSE: 0.51 |         Val. Loss: 0.35 | Val. MSE: 0.35 |  Val. RMSE: 0.59 |
| Epoch: 64 | Train Loss: 0.26 | Train MSE: 0.26 | Train RMSE: 0.51 |         Val. Loss: 0.35 | Val. MSE: 0.35 |  Val. RMSE: 0.59 |
| Epoch: 65 | Train Loss: 0.26 | Train MSE: 0.26 | Train RMSE: 0.51 |         Val. Loss: 0.34 | Val. MSE: 0.34 |  Val. RMSE: 0.58 |
| Epoch: 66 | Train Loss: 0.26 | Train MSE: 0.26 | Train RMSE: 0.51 |         Val. Loss: 0.34 | Val. MSE: 0.34 |  Val. RMSE: 0.59 |
| Epoch: 67 | Train Loss: 0.26 | Train MSE: 0.26 | Train RMSE: 0.51 |         Val. Loss: 0.35 | Val. MSE: 0.35 |  Val. RMSE: 0.59 |
| Epoch: 68 | Train Loss: 0.26 | Train MSE: 0.26 | Train RMSE: 0.51 |         Val. Loss: 0.34 | Val. MSE: 0.34 |  Val. RMSE: 0.59 |
| Epoch: 69 | Train Loss: 0.26 | Train MSE: 0.26 | Train RMSE: 0.51 |         Val. Loss: 0.34 | Val. MSE: 0.34 |  Val. RMSE: 0.58 |
| Epoch: 70 | Train Loss: 0.26 | Train MSE: 0.26 | Train RMSE: 0.51 |       

| Epoch: 125 | Train Loss: 0.25 | Train MSE: 0.25 | Train RMSE: 0.50 |         Val. Loss: 0.35 | Val. MSE: 0.35 |  Val. RMSE: 0.59 |
| Epoch: 126 | Train Loss: 0.25 | Train MSE: 0.25 | Train RMSE: 0.50 |         Val. Loss: 0.35 | Val. MSE: 0.35 |  Val. RMSE: 0.59 |
| Epoch: 127 | Train Loss: 0.26 | Train MSE: 0.26 | Train RMSE: 0.51 |         Val. Loss: 0.35 | Val. MSE: 0.35 |  Val. RMSE: 0.59 |
| Epoch: 128 | Train Loss: 0.25 | Train MSE: 0.25 | Train RMSE: 0.50 |         Val. Loss: 0.35 | Val. MSE: 0.35 |  Val. RMSE: 0.59 |
| Epoch: 129 | Train Loss: 0.25 | Train MSE: 0.25 | Train RMSE: 0.50 |         Val. Loss: 0.35 | Val. MSE: 0.35 |  Val. RMSE: 0.59 |
| Epoch: 130 | Train Loss: 0.25 | Train MSE: 0.25 | Train RMSE: 0.50 |         Val. Loss: 0.35 | Val. MSE: 0.35 |  Val. RMSE: 0.59 |
| Epoch: 131 | Train Loss: 0.25 | Train MSE: 0.25 | Train RMSE: 0.50 |         Val. Loss: 0.35 | Val. MSE: 0.35 |  Val. RMSE: 0.60 |
| Epoch: 132 | Train Loss: 0.25 | Train MSE: 0.25 | Train RMSE: 0.50 

| Epoch: 187 | Train Loss: 0.25 | Train MSE: 0.25 | Train RMSE: 0.50 |         Val. Loss: 0.35 | Val. MSE: 0.35 |  Val. RMSE: 0.59 |
| Epoch: 188 | Train Loss: 0.25 | Train MSE: 0.25 | Train RMSE: 0.50 |         Val. Loss: 0.35 | Val. MSE: 0.35 |  Val. RMSE: 0.59 |
| Epoch: 189 | Train Loss: 0.25 | Train MSE: 0.25 | Train RMSE: 0.50 |         Val. Loss: 0.35 | Val. MSE: 0.35 |  Val. RMSE: 0.59 |
| Epoch: 190 | Train Loss: 0.25 | Train MSE: 0.25 | Train RMSE: 0.50 |         Val. Loss: 0.35 | Val. MSE: 0.35 |  Val. RMSE: 0.59 |
| Epoch: 191 | Train Loss: 0.25 | Train MSE: 0.25 | Train RMSE: 0.50 |         Val. Loss: 0.35 | Val. MSE: 0.35 |  Val. RMSE: 0.59 |
| Epoch: 192 | Train Loss: 0.25 | Train MSE: 0.25 | Train RMSE: 0.50 |         Val. Loss: 0.35 | Val. MSE: 0.35 |  Val. RMSE: 0.59 |
| Epoch: 193 | Train Loss: 0.25 | Train MSE: 0.25 | Train RMSE: 0.50 |         Val. Loss: 0.35 | Val. MSE: 0.35 |  Val. RMSE: 0.59 |
| Epoch: 194 | Train Loss: 0.25 | Train MSE: 0.25 | Train RMSE: 0.50 